In [1]:
import dask.dataframe as dd
import pandas as pd
import os
import yaml

from dask.distributed import Client

Good; and in this case, Notebook is a good choice for your tooling.

In [62]:
client = Client(n_workers = 16)

In [63]:
# client.shutdown()

In [86]:
# load the data

#make a config file

with open('../config.yaml','r') as stream:
    config = yaml.safe_load(stream)
file_path = config['bacilli']

df = dd.read_csv(file_path, delimiter = "\t")
df

,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclodehydratase domain,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-.1
npartitions=174,,,,,,,,,,,,,,,
,object,object,int64,object,object,object,int64,int64,float64,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [66]:
df.shape

(Delayed('int-93d63cbc-c048-48aa-8a63-b0efc7f1d607'), 15)

In [67]:
# Change col names and type of one column
dtypes = {'score': object}
colnames = ['protein_accesion', 'MD5', 'length', 'analysis', 'signature_accession', 'signature_description', 'start_loc',
'stop_loc', 'score', 'status', 'date', 'ip_annot_accession', 'ip_annot_desc', 'GO_annotation', 'pathway']
df = dd.read_csv(file_path, delimiter="\t", names=colnames,dtype = dtypes)
df

,protein_accesion,MD5,length,analysis,signature_accession,signature_description,start_loc,stop_loc,score,status,date,ip_annot_accession,ip_annot_desc,GO_annotation,pathway
npartitions=174,,,,,,,,,,,,,,,
,object,object,int64,object,object,object,int64,int64,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [68]:
df.head(2)

,protein_accesion,MD5,length,analysis,signature_accession,signature_description,start_loc,stop_loc,score,status,date,ip_annot_accession,ip_annot_desc,GO_annotation,pathway
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-


In [9]:
# Q1 = How many distinct protein annotations are found in the dataset? 
# I.e. how many distinc InterPRO numbers are there?

distinct_annotations = df['ip_annot_accession'].nunique().compute(num_workers=16)
print("Number of distinct protein annotations: ", distinct_annotations)

Number of distinct protein annotations:  9704


In [10]:
# Q2: How many annotations does a protein have on average?

average_annotations_per_protein = df.groupby('protein_accesion').size().mean().compute(num_workers=16)
print("Average number of annotations per protein:", average_annotations_per_protein)

Average number of annotations per protein: 11.490524386574391


In [11]:
# Q3: What is the most common GO Term found?

most_common_go_term = df['GO_annotation'].value_counts().idxmax().compute(num_workers=16)
print("Most common GO Term found: ", most_common_go_term)

Most common GO Term found:  -


In [12]:
# I wanted to find the exact value of GO Term, So I try to find 2 most common:
# nlargest(2): the 2 most common GO Term
most_common_go_terms = df['GO_annotation'].value_counts().nlargest(2).compute(num_workers=16)
most_common_go_terms

-                        945977
GO:0003700|GO:0006355     33500
Name: GO_annotation, dtype: int64

In [13]:
# Q4: What is the average size of an InterPRO feature found in the dataset?

average_size = df['length'].mean().compute(num_workers=16)
print("Average size of an InterPRO feature: ", average_size)

Average size of an InterPRO feature:  408.1892178981481


In [14]:
# Q5: What is the top 10 most common InterPRO features?
top_10_common = df['ip_annot_accession'].value_counts().nlargest(10).compute(num_workers=16)
print("Top 10 most common InterPRO features:")
print(top_10_common)


Top 10 most common InterPRO features:
-            2278774
IPR027417      46834
IPR002347      18077
IPR003439      16944
IPR036388      16291
IPR036259      12602
IPR003593      11521
IPR036390      11304
IPR036291      10716
IPR000515      10613
Name: ip_annot_accession, dtype: int64


In [69]:
# Q6: If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, 
# what is the top10 then?

protein_size = df['length']

filtered_interpro_features = df[abs(df['length'] - protein_size) <= 0.1 * protein_size]
top_10_filtered_interpro_features = filtered_interpro_features['ip_annot_accession'].value_counts().nlargest(10).compute()
top_10_filtered_interpro_features

-            2278774
IPR027417      46834
IPR002347      18077
IPR003439      16944
IPR036388      16291
IPR036259      12602
IPR003593      11521
IPR036390      11304
IPR036291      10716
IPR000515      10613
Name: ip_annot_accession, dtype: int64

In [20]:
# Q7: If you look at those features which also have textual annotation, 
#what is the top 10 most common word found in that annotation?
import re
from collections import Counter

# Extract the textual annotation column from the filtered DataFrame
textual_annotations = filtered_interpro_features['ip_annot_desc'].compute(num_workers=16)

# Combine all the textual annotations into a single string
combined_text = ' '.join(textual_annotations)

# Remove non-alphanumeric characters and convert to lowercase
cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', combined_text.lower())

# Split the cleaned text into individual words
words = cleaned_text.split()

# Count the occurrences of each word
word_counts = Counter(words)

# Get the top 10 most common words
top_10_common_words = word_counts.most_common(10)

print("Top 10 most common words in the textual annotations:")
for word, count in top_10_common_words:
    print(word, count)

Top 10 most common words in the textual annotations:
domain 663290
superfamily 324250
protein 253203
cterminal 102120
hydrolase 87803
site 74724
dnabinding 65120
nterminal 64509
subunit 62524
type 59580


In [22]:
# Q8: And the top 10 least common?

import re
from collections import Counter

# Extract the textual annotation column from the filtered DataFrame
textual_annotations = filtered_interpro_features['ip_annot_desc'].compute(num_workers=16)

# Combine all the textual annotations into a single string
combined_text = ' '.join(textual_annotations)

# Remove non-alphanumeric characters and convert to lowercase
cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', combined_text.lower())

# Split the cleaned text into individual words
words = cleaned_text.split()

# Count the occurrences of each word
word_counts = Counter(words)

# Get the top 10 least common words
top_10_least_common_words = word_counts.most_common()[:-11:-1]

print("Top 10 least common words in the textual annotations:")
for word, count in top_10_least_common_words:
    print(word, count)

Top 10 least common words in the textual annotations:
eco47ii 1
hopa1 1
g39 1
duf5659 1
moxr 1
duf3037 1
bromoperoxidasechloroperoxidase 1
duf2269 1
bacteriophageassociated 1
fus7luc3like 1


In [70]:
# Q9: Combining your answers for Q6 and Q7, 
#what are the 10 most commons words found for the largest InterPRO features?

import re
from collections import Counter

# Filter the DataFrame to include InterPRO features that are almost the same size as the protein
protein_size = df['length']
filtered_interpro_features = df[abs(df['length'] - protein_size) <= 0.1 * protein_size]

# Extract the textual annotation column from the filtered DataFrame
textual_annotations = filtered_interpro_features['ip_annot_desc'].compute(num_workers=16)

# Combine all the textual annotations into a single string
combined_text = ' '.join(textual_annotations)

# Remove non-alphanumeric characters and convert to lowercase
cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', combined_text.lower())

# Split the cleaned text into individual words
words = cleaned_text.split()

# Count the occurrences of each word
word_counts = Counter(words)

# Get the 10 most common words
top_10_common_words = word_counts.most_common(10)

print("10 most common words for the largest InterPRO features:")
for word, count in top_10_common_words:
    print(word, count)


10 most common words for the largest InterPRO features:
domain 663290
superfamily 324250
protein 253203
cterminal 102120
hydrolase 87803
site 74724
dnabinding 65120
nterminal 64509
subunit 62524
type 59580


In [83]:
# Q10: What is the coefficient of correlation () between the size of the protein and the number of features found?
import numpy as np

new_df = filtered_interpro_features.groupby(['protein_accesion','length']).size().compute(num_workers=16)
new_df = new_df.reset_index()
new_df = new_df.rename(columns={0: 'num_features'})

# Filter the DataFrame to include InterPRO features that are almost the same size as the protein
protein_size = new_df['length']

# Count the number of features found for each protein accession
num_features = new_df['num_features']

print(protein_size.shape)
print(num_features.shape)

(365570,)
(365570,)


In [84]:
correlation_coefficient = np.corrcoef(protein_size, num_features)[0, 1]

print("Correlation coefficient between protein size and the number of features found:", correlation_coefficient)


Correlation coefficient between protein size and the number of features found: 0.6387083370361596
